In [ ]:
!pip install -U sentence-transformers
!pip install datasets
!pip install transformers
!pip install -U scikit-learn
!pip install InstructorEmbedding

In [ ]:
from datasets import load_dataset_builder
ds_builder = load_dataset_builder("imdb")

In [ ]:
ds_builder.info.description

'Large Movie Review Dataset.\nThis is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.'

In [ ]:
ds_builder.info.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [ ]:
from datasets import load_dataset

train = load_dataset("imdb", split="train")
train

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
test = load_dataset("imdb", split="test")
test

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

# **Sentence Transformer**

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
sentence_embeddings = model.encode(train['text'])

In [ ]:
sentence_embeddings.shape

(25000, 768)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, max_iter=1000).fit(sentence_embeddings, train['label'])

In [ ]:
clf.score(sentence_embeddings, train['label'])

0.89856

In [ ]:
test_embeddings = model.encode(test['text'])

In [ ]:
clf.score(test_embeddings, test['label'])

0.89152

# **instructor-xl**

In [ ]:
from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-xl')

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
def get_embedding(text):
    instruction = "Represent the movie sentiment for classification: "
    return model.encode([[instruction,text]])

In [ ]:
embeddings = list(map(get_embedding, train["text"]))

In [ ]:
import numpy as np
embds = np.concatenate(embeddings)

In [ ]:
embds.shape

(25000, 768)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, max_iter=1000).fit(embds, train['label'])

In [ ]:
clf.score(embds, train['label'])

0.93944

In [ ]:
test_embeddings = list(map(get_embedding, test['text']))

In [ ]:
test_embds = np.concatenate(test_embeddings)

In [ ]:
clf.score(test_embds, test['label'])

0.93872